In [1]:
# !pip install ipython_sql
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [2]:
%sql sqlite:////DS/Github/Springboard_DSC/SQLFiles_Tier_2/sqlite_db_pythonsqlite.db

In [3]:
# test
%sql select * from Facilities limit 3

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50


In [4]:
%sql select * from Bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


In [5]:
%%sql select * 
from Members 
limit 3;

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23


In [6]:
import sqlite3
import pandas as pd

con = sqlite3.connect('sqlite_db_pythonsqlite.db')

# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!
df0 = pd.read_sql("select * from (select name, case when sum(revenue)<1000 then sum(revenue) else NULL end as TotalRevenue from (select name, case when b.memid=0 then f.guestcost * b.slots else f.membercost * b.slots end as revenue\
                 from Facilities as f left join Bookings as b on f.facid=b.facid) group by name) where TotalRevenue is not null order by TotalRevenue", con)
df0
# cur = con.cursor()
# cur.execute("select name, TotalRevenue from (select name, sum(revenue) as TotoalRevenue from (select name, case when b.memid=0 then f.guestcost * b.slots else f.membercost * b.slots end as revenue\
#                  from Facilities as f left join Bookings as b on f.facid=b.facid) group by name) where TotalRevenue<1000 order by TotalRevenue")
# print(cur.fetchall())
#con.close()

,name,TotalRevenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [7]:
%%sql
select * 
from (select name, case when sum(revenue)<1000 then sum(revenue) end as TotalRevenue 
      from (select name, 
            case when b.memid=0 then f.guestcost * b.slots 
            else f.membercost * b.slots end as revenue 
            from Facilities as f 
            left join Bookings as b 
            on f.facid=b.facid) 
      group by name) 
where TotalRevenue is not null 
order by TotalRevenue  

name,TotalRevenue
Table Tennis,180
Snooker Table,240
Pool Table,270


In [8]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
df1 = pd.read_sql("select A.firstname as cust_firstname,A.surname as cust_surname,B.firstname as remd_firstname,B.surname as remd_surname from Members A, Members B\
                  where A.recommendedby >= 0 and A.recommendedby = B.memid order by cust_surname, cust_firstname", con)
df1

,cust_firstname,cust_surname,remd_firstname,remd_surname
0,Florence,Bader,Ponder,Stibbons
1,Anne,Baker,Ponder,Stibbons
2,Timothy,Baker,Jemima,Farrell
3,Tim,Boothe,Tim,Rownam
4,Gerald,Butters,Darren,Smith
5,Joan,Coplin,Timothy,Baker
6,Erica,Crumpet,Tracy,Smith
7,Nancy,Dare,Janice,Joplette
8,Matthew,Genting,Gerald,Butters
9,John,Hunt,Millicent,Purview


In [9]:
%%sql
select A.firstname as cust_firstname,A.surname as cust_surname,
B.firstname as remd_firstname,B.surname as remd_surname 
from Members A, Members B 
where A.recommendedby >= 0 and A.recommendedby = B.memid 
order by cust_surname, cust_firstname

cust_firstname,cust_surname,remd_firstname,remd_surname
Florence,Bader,Ponder,Stibbons
Anne,Baker,Ponder,Stibbons
Timothy,Baker,Jemima,Farrell
Tim,Boothe,Tim,Rownam
Gerald,Butters,Darren,Smith
Joan,Coplin,Timothy,Baker
Erica,Crumpet,Tracy,Smith
Nancy,Dare,Janice,Joplette
Matthew,Genting,Gerald,Butters
John,Hunt,Millicent,Purview


In [10]:
# Q12: Find the facilities with their usage by member, but not guests
df2 = pd.read_sql("select f.name,sum(b.slots) as usage from Facilities as f left join Bookings as b on f.facid=b.facid where b.memid != 0 group by f.name", con)
df2

,name,usage
0,Badminton Court,1086
1,Massage Room 1,884
2,Massage Room 2,54
3,Pool Table,856
4,Snooker Table,860
5,Squash Court,418
6,Table Tennis,794
7,Tennis Court 1,957
8,Tennis Court 2,882


In [11]:
%%sql
select f.name,sum(b.slots) as usage 
from Facilities as f 
left join Bookings as b 
on f.facid=b.facid 
where b.memid != 0 
group by f.name

name,usage
Badminton Court,1086
Massage Room 1,884
Massage Room 2,54
Pool Table,856
Snooker Table,860
Squash Court,418
Table Tennis,794
Tennis Court 1,957
Tennis Court 2,882


In [12]:
# Q13: Find the facilities usage by month, but not guests
df3 = pd.read_sql("select f.name, month, sum(slots) from Facilities as f \
                  left join \
                  (SELECT facid, slots, memid, \
                  substr(starttime,6,2) as month FROM Bookings) as sub \
                  on f.facid = sub.facid where memid != 0 group by name, month", con)
df3

,name,month,sum(slots)
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110


In [13]:
%%sql 
select f.name, month, sum(slots) 
from Facilities as f 
left join (SELECT facid, slots, memid, 
           substr(starttime,6,2) as month 
           FROM Bookings) as sub 
on f.facid = sub.facid 
where memid != 0 
group by name, month

name,month,sum(slots)
Badminton Court,07,165
Badminton Court,08,414
Badminton Court,09,507
Massage Room 1,07,166
Massage Room 1,08,316
Massage Room 1,09,402
Massage Room 2,07,8
Massage Room 2,08,18
Massage Room 2,09,28
Pool Table,07,110
